**Домашнее задание к уроку 2: Линейная и логистическая регрессия**\
**Цель задания**\
**Закрепить навыки работы с PyTorch API, изучить модификацию моделей и работу с различными датасетами.**\
**Задание 1: Модификация существующих моделей (30 баллов)**\
**1.1 Расширение линейной регрессии (15 баллов)** 

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from utils import make_regression_data, mse, log_epoch, RegressionDataset

class LinearRegression(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.linear = nn.Linear(in_features, 1)

    def forward(self, x):
        return self.linear(x)

if __name__ == '__main__':
    # Генерируем данные
    X, y = make_regression_data(n=200)
    
    # Создаём датасет и даталоадер
    dataset = RegressionDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    print(f'Размер датасета: {len(dataset)}')
    print(f'Количество батчей: {len(dataloader)}')
    
    # Создаём модель, функцию потерь и оптимизатор
    model = LinearRegression(in_features=1)
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1)
    
    reg_type = 'l2'  # или 'l1', или None
    alpha = 0.0001    # Коэффициент регуляризации

    d = 0.0001 #коэфицент остановки
    prev_weights = [w.detach().clone() for w in model.linear.weight] #сохраняем веса 

    # Обучаем модель
    epochs = 100
    for epoch in range(1, epochs + 1):
        total_loss = 0
        
        for i, (batch_X, batch_y) in enumerate(dataloader):
            optimizer.zero_grad()
            y_pred = model(batch_X)
            reg = 0
            if reg_type == 'l1':
                reg = alpha * model.linear.weight.abs().sum() #l1
            elif reg_type == 'l2':
                reg = alpha * (model.linear.weight ** 2).sum() #l2
            loss = criterion(y_pred, batch_y) + reg #штраф
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()

        weight_change = ((model.linear.weight - prev_weights[0]) ** 2).sum().sqrt().item() #проверка разницы весов с прошлой итерацией
        if weight_change < d:
            print(f"Остановка перед переобучением")
            break
        prev_weights = [w.detach().clone() for w in model.linear.weight] #сохраняем веса итерации

        avg_loss = total_loss / (i + 1)
        if epoch % 10 == 0:
            log_epoch(epoch, avg_loss)

    # Сохраняем модель
    torch.save(model.state_dict(), 'linreg_torch.pth')
    
    # Загружаем модель
    new_model = LinearRegression(in_features=1)
    new_model.load_state_dict(torch.load('linreg_torch.pth'))
    new_model.eval() 

Размер датасета: 200
Количество батчей: 7
Epoch 10: loss=0.0370
Epoch 20: loss=0.0155
Epoch 30: loss=0.0112
Epoch 40: loss=0.0113
Epoch 50: loss=0.0108
Epoch 60: loss=0.0125
Epoch 70: loss=0.0104
Остановка перед переобучением


**1.2 Расширение логистической регрессии (15 баллов)**

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from utils import make_classification_data, accuracy, log_epoch, ClassificationDataset, metrics
import warnings
from sklearn.exceptions import UndefinedMetricWarning
from sklearn.metrics import confusion_matrix
import numpy as np
import torch.nn.functional as F
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
class LogisticRegression(nn.Module):
    def __init__(self, in_features,num_classes):
        super().__init__()
        self.linear = nn.Linear(in_features, num_classes) #добавили многоклассовость

    def forward(self, x):
        return self.linear(x)

if __name__ == '__main__':
    # Генерируем данные
    num_classes = 3
    X, y = make_classification_data(n=200, num_classes=num_classes) #подаем колво классов
    
    # Создаём датасет и даталоадер
    dataset = ClassificationDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    print(f'Размер датасета: {len(dataset)}')
    print(f'Количество батчей: {len(dataloader)}')
    
    # Создаём модель, функцию потерь и оптимизатор
    model = LogisticRegression(in_features=X.shape[1], num_classes=num_classes) #изменили модель
    criterion = nn.CrossEntropyLoss() #поменяли loss функцию для многоклассовой
    optimizer = optim.SGD(model.parameters(), lr=0.1)
    
    # Обучаем модель
    epochs = 100
    for epoch in range(1, epochs + 1):
        all_preds = []
        all_true = []
        total_loss = 0
        total_acc = 0
        
        for i, (batch_X, batch_y) in enumerate(dataloader):
            optimizer.zero_grad()
            logits = model(batch_X)

            y_probs = torch.softmax(logits, dim=1) # Преобразуем логиты в вероятности
            y_pred = y_probs.argmax(dim=1) # Получаем предсказанные классы
            all_preds.extend(y_pred.detach().cpu().numpy()) # Сохраняем предсказания для всей эпохи
            all_true.extend(batch_y.detach().cpu().numpy()) # Сохраняем истинные метки для всей эпохи

            loss = criterion(logits, batch_y)
            loss.backward()
            optimizer.step()
            
            # Вычисляем accuracy
            y_pred = logits
            acc = accuracy(y_pred, batch_y)
            
            total_loss += loss.item()
            total_acc += acc
        
        avg_loss = total_loss / (i + 1)
        avg_acc = total_acc / (i + 1)
        
        precision, recall, f1, roc_auc = metrics(logits, batch_y) #нахождение метрик


        if epoch % 10 == 0:
            log_epoch(epoch, avg_loss, acc=avg_acc)
            print(
            f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, ROC-AUC: {roc_auc:.4f}')

    # Визуализация confusion matrix
    cm = confusion_matrix(all_true, all_preds)  # Вычисление confusion matrix

    num_classes = cm.shape[0]
    class_names = [f"{i}" for i in range(num_classes)]

    print("\nConfusion Matrix:")
    header = "Pred → " + "  ".join([f"{name:>6}" for name in class_names])
    print(header)
    for i, row in enumerate(cm):
        row_str = " ".join([f"{val:>6}" for val in row])
        print(f"True {class_names[i]:>3} {row_str}")
    
    # Сохраняем модель
    torch.save(model.state_dict(), 'logreg_torch.pth')
    
    # Загружаем модель
    new_model = LogisticRegression(in_features=X.shape[1], num_classes=num_classes)
    new_model.load_state_dict(torch.load('logreg_torch.pth'))
    new_model.eval()

Размер датасета: 200
Количество батчей: 7
Epoch 10: loss=0.6445, acc=0.7500
Precision: 0.6667, Recall: 0.5833, F1: 0.6190, ROC-AUC: nan
Epoch 20: loss=0.6381, acc=0.7188
Precision: 0.8056, Recall: 0.7778, F1: 0.7190, ROC-AUC: 0.8389
Epoch 30: loss=0.6704, acc=0.7098
Precision: 0.5556, Recall: 0.5000, F1: 0.5238, ROC-AUC: 0.8006
Epoch 40: loss=0.6662, acc=0.7143
Precision: 0.2778, Recall: 0.2778, F1: 0.2778, ROC-AUC: nan
Epoch 50: loss=0.6001, acc=0.7545
Precision: 0.9333, Recall: 0.8889, F1: 0.8963, ROC-AUC: 1.0000
Epoch 60: loss=0.6309, acc=0.7411
Precision: 0.7667, Recall: 0.7778, F1: 0.6852, ROC-AUC: 0.8190
Epoch 70: loss=0.6353, acc=0.7143
Precision: 0.6333, Recall: 0.5000, F1: 0.5000, ROC-AUC: 0.8278
Epoch 80: loss=0.6002, acc=0.7455
Precision: 0.9167, Recall: 0.9167, F1: 0.9048, ROC-AUC: 1.0000
Epoch 90: loss=0.6171, acc=0.7545
Precision: 0.9167, Recall: 0.8889, F1: 0.8857, ROC-AUC: 0.8889
Epoch 100: loss=0.6305, acc=0.7366
Precision: 0.7333, Recall: 0.7333, F1: 0.7333, ROC-AUC: 